In [12]:
import pandas as pd
import numpy as np

In [1]:
df_train = pd.read_csv("data/train.csv")
X = df_train.iloc[:, :-1].values
y = df_train.iloc[:, -1].values

df_kaggle = pd.read_csv("data/test.csv")
X_kaggle = df_kaggle.iloc[:, :].values

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder

# ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(handle_unknown="ignore"), [0, 1])], remainder="passthrough")
# X = np.array(ct.fit_transform(X))
# X_kaggle = np.array(ct.transform(X_kaggle))

In [2]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X
y_train = y

In [ ]:
c = "tipo_vendedor"
v = 1
print("Média:",df_train.loc[df_train[c] == v]["preco"].mean())
print("Mediana:",df_train.loc[df_train[c] == v]["preco"].median())
print("Desvio Padrão:",df_train.loc[df_train[c] == v]["preco"].std())
print("Min:",min(df_train.loc[df_train[c] == v]["preco"]))
print("Max:",max(df_train.loc[df_train[c] == v]["preco"]))
df_train.loc[df_train[c] == v]

In [ ]:
df_train.sort_values(by="preco", ascending=True).iloc[3:-3]

list(df_train).index("quartos")

In [3]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train[:, 84:89] = sc.fit_transform(X_train[:, 84:89])

sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1))

# sc_y = StandardScaler()
# y_train = sc_y.fit_transform(y_train.reshape(-1,1))

sc_kaggle = StandardScaler()
X_kaggle[:, 83:88] = sc_kaggle.fit_transform(X_kaggle[:, 83:88])

In [4]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

clf = CatBoostRegressor(silent=True)
params = {'depth'         : [6,8,10],
          'learning_rate' : [0.01, 0.05, 0.1],
          'iterations'    : [30, 50, 100]
         }
scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

clf_grid.fit(X_train, y_train)
best_param = clf_grid.best_params_
best_param

C:\Development\Python3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Development\Python3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\Development\Python3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_classification.py", line 202, in accuracy_score
    y_type, y_true, y_pred = _ch

{'depth': 6, 'iterations': 30, 'learning_rate': 0.01}

In [5]:
best_param

{'depth': 6, 'iterations': 30, 'learning_rate': 0.01}

## Utilização de *Gradient Boosting* com a biblioteca *CatBoost* para treinar modelo de regressão

In [6]:
from catboost import CatBoostRegressor

regressor = CatBoostRegressor(depth=6, iterations=30, learning_rate=0.01)
regressor.fit(X_train, y_train)

0:	learn: 0.9980426	total: 1.71ms	remaining: 49.5ms
1:	learn: 0.9960968	total: 3.21ms	remaining: 44.9ms
2:	learn: 0.9941589	total: 4.44ms	remaining: 40ms
3:	learn: 0.9922202	total: 5.82ms	remaining: 37.9ms
4:	learn: 0.9902949	total: 7.21ms	remaining: 36.1ms
5:	learn: 0.9883723	total: 8.57ms	remaining: 34.3ms
6:	learn: 0.9864521	total: 9.95ms	remaining: 32.7ms
7:	learn: 0.9845309	total: 11.3ms	remaining: 31.2ms
8:	learn: 0.9826256	total: 12.7ms	remaining: 29.6ms
9:	learn: 0.9807249	total: 14ms	remaining: 28ms
10:	learn: 0.9788202	total: 15.5ms	remaining: 26.7ms
11:	learn: 0.9769386	total: 16.6ms	remaining: 25ms
12:	learn: 0.9750626	total: 18ms	remaining: 23.5ms
13:	learn: 0.9731877	total: 19.3ms	remaining: 22.1ms
14:	learn: 0.9713204	total: 20.4ms	remaining: 20.4ms
15:	learn: 0.9694626	total: 21.8ms	remaining: 19ms
16:	learn: 0.9676110	total: 23ms	remaining: 17.6ms
17:	learn: 0.9657590	total: 24.4ms	remaining: 16.2ms
18:	learn: 0.9639206	total: 25.7ms	remaining: 14.9ms
19:	learn: 0.9620

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix

# y_pred = classifier.predict(X_test)
# acc_sc = accuracy_score(y_test, y_pred)
# cf_mx = confusion_matrix(y_test, y_pred)
# print("Accuracy Score:", acc_sc)
# print(cf_mx)

In [7]:
from sklearn.model_selection import cross_val_score
from datetime import datetime as dt
timestamp = dt.today()

accuracies = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10)
print(f"Accuracy: {accuracies.mean()}")
print(f"StdDev: {accuracies.std()}")
try:
    with open("data/cross_val.log", mode="x+") as cvlog:
        cvlog.write("timestamp,accuracy,stddev")
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")
except:
    with open("data/cross_val.log", mode="a") as cvlog:
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")

0:	learn: 0.5084973	total: 7.27ms	remaining: 211ms
1:	learn: 0.5072828	total: 8.72ms	remaining: 122ms
2:	learn: 0.5060286	total: 10.2ms	remaining: 91.9ms
3:	learn: 0.5047962	total: 11.6ms	remaining: 75.3ms
4:	learn: 0.5035921	total: 13ms	remaining: 65.1ms
5:	learn: 0.5023912	total: 14.4ms	remaining: 57.6ms
6:	learn: 0.5011909	total: 15.7ms	remaining: 51.7ms
7:	learn: 0.4999948	total: 17ms	remaining: 46.8ms
8:	learn: 0.4987786	total: 18.3ms	remaining: 42.8ms
9:	learn: 0.4975898	total: 19.7ms	remaining: 39.4ms
10:	learn: 0.4964043	total: 21ms	remaining: 36.3ms
11:	learn: 0.4952224	total: 22.2ms	remaining: 33.2ms
12:	learn: 0.4940431	total: 23.6ms	remaining: 30.9ms
13:	learn: 0.4928315	total: 25.2ms	remaining: 28.8ms
14:	learn: 0.4916403	total: 26.8ms	remaining: 26.8ms
15:	learn: 0.4904708	total: 28.4ms	remaining: 24.8ms
16:	learn: 0.4893043	total: 29.8ms	remaining: 22.8ms
17:	learn: 0.4881407	total: 31.2ms	remaining: 20.8ms
18:	learn: 0.4869799	total: 32.7ms	remaining: 18.9ms
19:	learn: 

In [8]:
y_kaggle = sc_y.inverse_transform(regressor.predict(X_kaggle))

In [9]:
with open(f"data/kaggle/kaggle_{timestamp.strftime('%d-%m-%Y_%Hh%M')}.csv", mode="w") as out:
    out.write("Id,preco")
    for i in range(len(y_kaggle)):
        out.write(f"\n{i},{round(y_kaggle[i], 2):.2f}")